# TP Métaheuristiques

## 1ère étape

### Lecture des jeux de données

https://homepages.laas.fr/huguet/drupal/content/métaheuristiques-4ir-info#overlay-context=content/m%25C3%25A9ta-heuristiques-4ir-info  

*Remarque* : Je pense qu'on peut enlever les arcs inutiles, en vérifiant si ils sont dans un des chemins d'évacuation des noeudsE.
    --> Penser alors à modifier la valeur de graphe.nombreArc ..

In [3]:
# Classes

class Evacuation:
    # Liste des noeuds à évacuer
    # Nombre de noeuds à évacuer
    # Le noeud sur
    
    def __init__(self):
        self.listeNoeudsE = []
    def sur(self, c):
        self.nombreNoeudsE = int(c[0])
        self.noeudSur = int(c[1])
    def ajoutNoeudE(self, c):
        nouveauNoeudE = NoeudAEvacuer(c)
        self.listeNoeudsE.append(nouveauNoeudE)
        
class NoeudAEvacuer:
    # ID/Population/taux d'évacuation max
    # Chemin d'évacuation, composé des id des noeuds par lesquels passer
    def __init__(self, c):
        self.nom = int(c[0])
        self.pop = int(c[1])
        self.tauxE = int(c[2])
        self.cheminE = [int(noeudId) for noeudId in c[4:]]
        
class Graphe: 
    # Liste d'arcs UTILES
    # Nombre d'arcs UTILES
    def __init__(self):
        self.listeArcs = []
    def nombre(self,c):
        self.nombreArcs = int(c[1])
    def ajoutArc(self, c):
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)
        # On ajoute le même arc avec ses bornes inversées.
        x = int(c[0])
        c[0] = int(c[1])
        c[1] = x
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)

class Arc: 
    # Arc origine/destination(int) ??  /Durée(int)/Capa(int) 
    def __init__(self, c):
        self.origine = int(c[0])
        self.destination = int(c[1])
        self.duree = int(c[3])
        self.capa = int(c[4])
        
# ------------------------------------

# Fonction principale

def lecture(nomFichier, evacuation, graphe): 
    with open("./Instances/" + nomFichier, "r") as fichier :
        print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        # Traitement de l'instance
        val = 1
        for ligne in lignes[1:]: 
            c = ligne.split()
            if c[0] == 'c': 
                val = 2
            else:
                if val == 1:
                    traitementEvacuation(c, evacuation)
                elif val == 2:
                    traitementGraphe(c, graphe)
                else :
                    sys.exit("[Erreur] lecture(nomFichier, arbre, graphe)")
                    
        # Suppression des arcs inutiles
        majArcsUtiles(evacuation, graphe)
        # Fermeture du fichier
        fichier.close()
        # Fin de la lecture

# ---------------------------------

# Fonctions annexes

def traitementEvacuation(c, evacuation):
    if len(c) != 2:
        evacuation.ajoutNoeudE(c)
    else:
        evacuation.sur(c)
        
def traitementGraphe(c, graphe):
    if len(c) != 2: 
        graphe.ajoutArc(c)
    else:
        graphe.nombre(c)

def majArcsUtiles(evacuation, graphe):    
    # Création d'une liste d'arcs vraiment utiles issus de l'arbre.
    arcsUtiles = []
    for noeud in evacuation.listeNoeudsE:
        arc = [noeud.nom, noeud.cheminE[0]]
        if arc not in arcsUtiles:
            arcsUtiles.append(arc)
        i = 0
        while True:
            arc = [noeud.cheminE[i], noeud.cheminE[i+1]]
            if arc not in arcsUtiles:
                arcsUtiles.append(arc)
            i = i + 1
            if(noeud.cheminE[i] == evacuation.noeudSur):
                break
        
    # Comparaison des arcs du graphe avec la liste arcsUtiles.
    nouveauxArcs = []
    for arcPossible in graphe.listeArcs:
        if [arcPossible.origine,arcPossible.destination] in arcsUtiles:
            nouveauxArcs.append(arcPossible)
            
    graphe.listeArcs = nouveauxArcs
    graphe.nombreArcs = len(graphe.listeArcs)

    
def affichage(evacuation, graphe):
    print("")
    print("=============================================")
    print("--- E V A C U A T I O N ---")
    print("Noeud sûr : %s." %(evacuation.noeudSur))
    print("Nombre de noeuds à évacuer : %s." %(evacuation.nombreNoeudsE)) 
    print("ID - Pop - TauxE (+ Chemin)")
    for noeudE in evacuation.listeNoeudsE:
        print("%s     %s     %s" %(noeudE.nom, noeudE.pop, noeudE.tauxE))
        print(*noeudE.cheminE, sep = " ==> ")
        print("------------------------------------")
    print("")
    print("--- A R B R E ---")
    print("Nombre d'arcs : %s." %(graphe.nombreArcs))
    print("Orig/Dest -- Durée -- Capacité")
    for arc in graphe.listeArcs:
        print("  %s/%s       %s      %s" %(arc.origine, arc.destination, arc.duree, arc.capa))
        print("------------------------------------")
    print("=============================================")    

Testons la méthode de lecture des données sur l'exemple du cours : 

In [4]:
def testLecture(): 
    evacuation = Evacuation()
    graphe = Graphe()

    lecture("exemple_cours.txt", evacuation, graphe)
    affichage(evacuation, graphe)

testLecture()

Name of the file: ./Instances/exemple_cours.txt

--- E V A C U A T I O N ---
Noeud sûr : 6.
Nombre de noeuds à évacuer : 3.
ID - Pop - TauxE (+ Chemin)
1     48     8
4 ==> 5 ==> 6
------------------------------------
2     30     5
4 ==> 5 ==> 6
------------------------------------
3     33     3
5 ==> 6
------------------------------------

--- A R B R E ---
Nombre d'arcs : 5.
Orig/Dest -- Durée -- Capacité
  1/4       7      8
------------------------------------
  2/4       4      5
------------------------------------
  3/5       6      3
------------------------------------
  4/5       9      10
------------------------------------
  5/6       12      11
------------------------------------


### Vérification et évaluation d’une solution

#### Données de l'exemple

cf exemple_cours.txt

#### Fichier solution de l'exemple

cf exemple_cours_solution.txt

#### Vérification de la solution

* Vérifier si la solution est réalisable = Respecte les contraintes.
    * vérifier à chaque instant t si la capacité de l'arc n'est pas dépassée = simulation
* Vérifier la valeur de la fonction objectif. Pour chaque noeud : 
    * trouver le nombre de personnes à évacuer
    * calculer la durée de transit des personnes de ce noeud : transit = noeudE.pop / solution.tauxE
    * calculer la durée du trajet du noeud à évacuer vers le noeud sécurisé : trajet = somme des durées des arcs du chemin d'évacuation
    * calculer la durée dévacuation de ce noeud : duree = transit + trajet

La valeur de la fonction objectif est le max des durées trouvées pour chaque noeud à évacuer. 

*Remarque* : Est-ce qu'il faut vérifier si la capacité dans le fichier des données est vraie ? Par exemple en parcourant le chemin d'évacuation, et en gardant le min des capacités. 

In [264]:
import numpy as np
import pandas as pd

# Fonction principale
    
def verification(nomFichier, nomFichierSolution, evacuation, graphe, solution):
    lecture_solution(nomFichierSolution, solution)
    lecture(solution.nom+".txt", evacuation, graphe)
    
    result = True
    
    # Vérifier la valeur de la fonction objectif
    # Pour chaque noeud à évacuer : 
    durees = []

    for noeudE in evacuation.listeNoeudsE: 
        # Trouver le nombre de personnes à évacuer 
        # Calculer la durée de transit des personnes de ce noeud : 
            # transit = noeudE.pop / solution.tauxE
        bonNoeudE = [noeudESol for noeudESol in solution.listeNoeudsESol if (noeudE.nom == noeudESol.nom)]
        transit = round(noeudE.pop / bonNoeudE[0].tauxE)
        # Calculer la durée du trajet du noeud à évacuer vers le noeud sécurisé : 
            # trajet = somme des durées des arcs du chemin d'évacuation
        origines = noeudE.cheminE[:-1]
        origines.append(noeudE.nom)
        trajet = round(sum([arc.duree for arc in graphe.listeArcs if (arc.origine in origines)]))
        
        # Calculer la durée dévacuation de ce noeud : duree = transit + trajet
        durees.append(transit + trajet + bonNoeudE[0].dateE)
    
    valFctObjectif = max(durees)
    
    if(valFctObjectif != solution.valFctObjectif):
        result = False
        print("\nValeur de la fonction objectif erronée : " + str(valFctObjectif) + " au lieu de " + str(solution.valFctObjectif) +".")
    else:   
    # Vérifier si la solution est réalisable
        print("\nValeur de la fonction objectif : " + str(result))
        # Génération du diagramme de Gantt  
        gantt = simulation(evacuation, graphe, solution, valFctObjectif)
        # On vérifie que la capacité de chaque arc est bien respectée
        affichage_gantt(gantt)
        simu = True
        for arc in graphe.listeArcs:
            if(gantt.loc[arc.origine, :] > arc.capa).any():
                simu = False
                result = False
        print("Simulation = " + str(simu))

    
    return result
        
# ---------------------------------

# Fonctions annexes

def lecture_solution(nomFichierSolution, solution): 
    with open("./Instances/" + nomFichierSolution, "r") as fichier :
        print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        # Traitement de l'instance
        solution.nom = lignes[0].split()[0]
        nbNoeudsE = int(lignes[1].split()[0])
        for ligne in lignes[2:(2+nbNoeudsE)]:
            c = ligne.split()
            solution.ajoutNoeudESol(c)
        solution.nature = lignes[2+nbNoeudsE].split()[0]
        solution.valFctObjectif = int(lignes[2+nbNoeudsE+1].split()[0])
        solution.tpsCalcul = int(lignes[2+nbNoeudsE+2].split()[0])
        solution.methode = " ".join(lignes[2+nbNoeudsE+3].split())
        solution.commentaire = " ".join(lignes[2+nbNoeudsE+4].split())
                                    
        # Fermeture du fichier
        fichier.close()

def affichage_solution(solution): 
    print("\n--- S O L U T I O N ---")
    print("Nom de l'instance résolue : " + solution.nom)
    print("Noeuds à évacuer : ")
    for noeud in solution.listeNoeudsESol:
        print("     ID : " + str(noeud.nom) + " - Taux d'évacuation : " + str(noeud.tauxE) + " - Date de début d'évacuation : " + str(noeud.dateE))
    print("Nature : " + solution.nature)
    print("Valeur de la fonction objectif : " + str(solution.valFctObjectif))
    print("Temps de calcul : " + str(solution.tpsCalcul))
    print("Méthode de résolution : " + solution.methode)
    print("Commentaire : " + solution.commentaire)
    print("")

def simulation(evacuation, graphe, solution, valFctObjectif): 
    result = True
    
    # On itialise 
    gantt = np.zeros((len(graphe.listeArcs), valFctObjectif-[arc.duree for arc in graphe.listeArcs if (arc.destination == evacuation.noeudSur)][0]), dtype=int)
    gantt = pd.DataFrame(gantt, index = [arc.origine for arc in graphe.listeArcs])

    # Pour chaque noeud à évacuer, on va remplir la capacité que l'on utilise sur chaque arc à chaque instant
    for noeudE in evacuation.listeNoeudsE: 
        bonNoeudE = [noeudESol for noeudESol in solution.listeNoeudsESol if (noeudE.nom == noeudESol.nom)][0]
        transit = round(noeudE.pop / bonNoeudE.tauxE)
        # On retrouve la durée de début d'évacuation
        time = [noeud.dateE for noeud in solution.listeNoeudsESol if (noeudE.nom == noeud.nom)][0]
        origines = noeudE.cheminE[:-1]
        origines.append(noeudE.nom)
        for arc in [arc for arc in graphe.listeArcs if (arc.origine in origines)]: 
            gantt.loc[arc.origine, time:(time+transit-1)] += bonNoeudE.tauxE
            time += arc.duree

    return gantt

def affichage_gantt(gantt):
    print("\n--- G A N T T ---")
    print(gantt)
    print("\n")

# ---------------------------------

# Classes

class NoeudAEvacuerSol: 
    # Identifiant
    # Taux d'évacuation
    # Date de début d'évacuation
    def __init__(self, c):
        self.nom = int(c[0])
        self.tauxE = int(c[1])
        self.dateE = int(c[2])

class Solution:
    # Nom de l'instance résolue
    # Pour chaque sommet à évacuer
        # Identifiant
        # Taux d'évacuation
        # Date de début d'évacuation
    # Nature de la solution (valid ou invalid)
    # Valeur de la fonction objectif
    # Temps de calcul
    # Méthode utilisée
    # Commentaire
    
    def __init__(self):
        self.listeNoeudsESol = []
    def ajoutNoeudESol(self, c):
        nouveauNoeudESol = NoeudAEvacuerSol(c)
        self.listeNoeudsESol.append(nouveauNoeudESol)

Testons la méthode de vérification d'une solution sur l'exemple du cours : 

In [269]:
def testVerification(nomFichier, nomFichierSolution): 
    evacuation = Evacuation()
    graphe = Graphe()
    solution = Solution()

    result = verification(nomFichier, nomFichierSolution, evacuation, graphe, solution)
    affichage_solution(solution)
    print("Solution vérifiée : " + str(result))

testVerification("exemple_cours.txt", "exemple_cours_solution.txt")

Name of the file: ./Instances/exemple_cours_solution.txt
Name of the file: ./Instances/exemple_cours.txt

Valeur de la fonction objectif : True

--- G A N T T ---
   0   1   2   3   4   5   6   7   8   9  ...  14  15  16  17  18  19  20  21  \
1   0   7   7   7   7   7   7   7   0   0 ...   0   0   0   0   0   0   0   0   
2   3   3   3   3   3   3   3   3   3   3 ...   0   0   0   0   0   0   0   0   
3   3   3   3   3   3   3   3   3   3   3 ...   0   0   0   0   0   0   0   0   
4   0   0   0   0   3   3   3   3  10  10 ...   7   0   0   0   0   0   0   0   
5   0   0   0   0   0   0   3   3   3   3 ...   6   6   6  10  10  10  10  10   

   22  23  
1   0   0  
2   0   0  
3   0   0  
4   0   0  
5  10   7  

[5 rows x 24 columns]


Simulation = True

--- S O L U T I O N ---
Nom de l'instance résolue : exemple_cours
Noeuds à évacuer : 
     ID : 1 - Taux d'évacuation : 7 - Date de début d'évacuation : 1
     ID : 2 - Taux d'évacuation : 3 - Date de début d'évacuation : 0
     ID : 